[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/How-to-Learn-to-Code/python-class/blob/master/Lesson_5_Pandas_DataFrame/Lesson5_pandas_DataFrame_Teacher.ipynb)

# Lesson 5: Pandas and DataFrames

## Learning Objectives:

Students will gain a better understanding of how to use `pandas` for manipulating data.

* [Introduction to `pandas`](#intro)
* [Loading and Slicing `DataFrame`s](#dataframes)
* [Joining `DataFrame`s](#joining)
* [Conditional Subsetting](#conditional)
* [Statistics and Plotting in `pandas`](#stats)
* [In-Class Exercises](#exercises)

### Introduction to `pandas` <a id='intro'></a>

In our last class, we discussed the basics of how to read and write files, allowing us to use any data we have. When we're using data that is structured in a specific way, it usually difficult to use and manipulate the text directly. Instead, some functions and modules implement an application programming interface (API) that allows us to better use the data.

In this lesson, we'll be downloading and using a CSV file (.csv), which stands for "comma separate value". If you look at this file's text, you'll see a tabular structure, resembling something like
```
day,temperature,wind_speed,humidity
1/1/2024,25,10,50
1/2/2024,26,12,55
1/3/2024,27,11,59
1/4/2024,28,13,54
```

Instead of using the data like this, we'll load in data with a module named `pandas` using the function `read_csv`. This will give us an object called a `DataFrame`. Basically, this is a two-dimensional table of rows and columns. 

This is useful in terms of allowing us to visualize our data, but most users will want to utilize the data for their own research purposes. Therefore, users will find it helpful to use other functions in `pandas` to help analyze or manipulate the data.

#### Setup

First, we need to install the `pandas` and some other libraries onto our Google Colab environment. The following cell will install the necessary libraries, but we can't quite use them yet.

In [ ]:
%pip install pandas 
%pip install matplotlib

In order to use the libraries, we need to import them. This will bring the libraries into our current notebook so that we can actively use them.

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt

For this lesson, we'll be using daily weather history data from multiple U.S. cities from July 2014 - June 2015 (from FiveThirtyEight's public repository of data found here: https://github.com/fivethirtyeight/data/tree/master/us-weather-history). The next cell will download the data and save it to files named `KPHL.csv` and `KSEA.csv`.

In [ ]:
!wget https://raw.githubusercontent.com/fivethirtyeight/data/master/us-weather-history/KPHL.csv
!wget https://raw.githubusercontent.com/fivethirtyeight/data/master/us-weather-history/KSEA.csv

### Loading and Slicing `DataFrame`s <a id='dataframes'></a>

The first set of data we are going to read in is from Philadelphia (`KPHL.csv`). We will use the `pandas` library to read in the data and display the first few rows of the data.

In [ ]:
philly_weather = pd.read_csv('KPHL.csv', index_col='date')
print(philly_weather.shape)
philly_weather.head()

As you can see, the rows of this data frame represent each day during this period, and the columns represent the weather data that has been collected.

As you can see, there are numerous types of data that has been collected, including the temperatures and amount of precipitation for that day and over the years.



#### Subsetting `DataFrame`s

Often times, our data may contain extra information that we don't need at the moment. We can subset the `DataFrame` to only include the columns that we need.

In `pandas`, one of the ways you can select certain subsets of the data is to specify them by the row and/or column names. Let's say we only want to look at the actual precipitation data. We can "take a slice" of or "index into" the `DataFrame` to only look at the precipitation data.

In [ ]:
philly_weather["actual_precipitation"]

We can grab multiple columns by passing in a list of column names. For example, if we want to look at the actual, average, and record precipitation data, we can pass in a list of the column names.

In [ ]:
philly_weather[["actual_precipitation", "average_precipitation", "record_precipitation"]]

Another way of selecting these columns is to use the `loc` method, which looks for the names of the columns by labels. Since we access the table rows by columns, we include `:` to say that we want all of the rows, but only the columns with specific labels. 

In [ ]:
philly_weather.loc[:, ["actual_precipitation", "average_precipitation", "record_precipitation"]]

Likewise, if we want specific rows, we can select the desired rows by their labels and use the colon `:` after a comma to get all the columns. 

In [ ]:
philly_weather.loc[["2014-7-1", "2014-7-2"], :]

Instead of just relying on the labels of the rows and/or columns, we can also use their numerical location or index. To do thisx, we can instead use the `iloc` method to select the row and/or column by index value(s). 

For instance, we can grab the first 3 rows (`0:2`) and the 10th-12th columns (`9:12`) like:

In [ ]:
philly_weather.iloc[0:2, 9:12]

If the numbers are not in a sequence (e.g., columns 0, 5, and 7), you can get the subset of the data using a list:

In [ ]:
philly_weather.iloc[:, [0, 5, 7]]

### Joining `DataFrame`s <a id='joining'></a>

Of course, we might not only be interested in the data from one city or file. Imagine that we want to do side-by-side comparisons of the average precipitation in Seattle vs. Philadelphia. 

Let's first read in the weather pattern data for Seattle (`KSEA.csv`) as well.

In [ ]:
seattle_weather = pd.read_csv('KSEA.csv', index_col='date')
seattle_weather.head()

Rather than having to flip between both data frames, we can instead combine them into one separate data frame using the `join` method. 

There are four different ways that we can join our data: the `left`, `right`, `inner`, and `outer` options.

For the sake demonstrating these different options, let's first subset our data to only look at the columns dealing with precipitation but grab 5 rows from each data frame. (We're using different rows for each `DataFrame` intentionally!) Then, let's sort our data with `sort_values` such that Philadelphia's data is sorted from the lowest to highest record precipitation, while Seattle's data is sorted from the highest to lowest record precipitation. 

In [ ]:
philly_weather_precip = philly_weather.iloc[:5, 9:12]
seattle_weather_precip = seattle_weather.iloc[1:6, 9:12]

In [ ]:
# Sort Philly weather from lowest to highest record precipitation
# Using sort_values:
# - by: column to sort by
# - axis: 0 to sort rows, 1 to sort columns
# - ascending: True to sort from low to high, False to sort from high to low
philly_weather_precip_reordered = philly_weather_precip.sort_values(by=["record_precipitation"], axis=0, ascending=True)
philly_weather_precip_reordered

In [ ]:
seattle_weather_precip_reordered = seattle_weather_precip.sort_values(by=["record_precipitation"], axis=0, ascending=False)
seattle_weather_precip_reordered

A `left` join (the default setting) indicates that we preserve the rows of the first `DataFrame` and their order, and then puts the `DataFrame`s next to each other. Notice that the `NaN` values are filled in for the missing data.

Since the column names are the same for both dataframes, we use `lsuffix` and `rsuffix` to rename and keep track of which columns came from which dataframe.

In [ ]:
# Notice that the Philly records are still increasing, but the Seattle records are out of order
philly_weather_precip_reordered.join(seattle_weather_precip_reordered, how='left', lsuffix='_philly', rsuffix='_seattle')

Likewise, a `right` join indicates that we preserve the rows of the second `DataFrame` and their order, and then puts the `DataFrame`s next to each other. Notice that the `NaN` values are filled in for the missing data.

In [ ]:
# Notice that the Seattle records are still decreasing, but the Philly records are out of order
philly_weather_precip_reordered.join(seattle_weather_precip_reordered, how='right', lsuffix='_philly', rsuffix='_seattle')

The `outer` join will combine the two `DataFrame`s taking all of the unique rows from both `DataFrame`s. If there is no data for a specific row, the `DataFrame` will fill in `NaN` values.

In [ ]:
# Notice that both records are out of order but there are 6 rows now.
philly_weather_precip_reordered.join(seattle_weather_precip_reordered, how='outer', lsuffix='_philly', rsuffix='_seattle')

The `inner` join will combine the two `DataFrame`s taking all of the rows from the `DataFrame`s such that there is no missing data.

In [ ]:
# Notice that both records are out of order but there are 4 rows now.
philly_weather_precip_reordered.join(seattle_weather_precip_reordered, how='inner', lsuffix='_philly', rsuffix='_seattle')

### Conditional Subsetting <a id='conditional'></a>

Another useful way to subset `DataFrame`s is with conditional subsetting. This allows us to filter the data based on certain conditions that we specify. Let's say that we're only interested in days where the max temperature was greater than 90 degrees. We can use the following code to filter the data:

In [ ]:
philly_weather[philly_weather["actual_max_temp"] > 90]

As another example, let's get all the days where there was no precipitation.

In [ ]:
philly_weather[philly_weather["actual_precipitation"] == 0.00]

### Statistics and Plotting in `pandas` <a id='stats'></a>

Let's make a new variable containing the combined Philly and Seattle precipitation data.

In [ ]:
philly_weather_precip = philly_weather.iloc[:, 9:12]
seattle_weather_precip = seattle_weather.iloc[:, 9:12]
all_precip = philly_weather_precip.join(seattle_weather_precip, how='left', lsuffix='_philly', rsuffix='_seattle')

We can perform statistics on our data as a whole using built-in mathematical functions such as `min`, `max`, or `mean`. These calculations can be performed on one or more columns of the `DataFrame`. 

For example, we can get the mean value of the actual precipitation in Seattle and Philadelphia:

In [ ]:
all_precip[["actual_precipitation_seattle", "actual_precipitation_philly"]].mean()

As you can see, on average the daily precipitation in Philadelphia is only 0.02 inches greater than Seattle. 

In general, if you wanted to look at multiple statistics at once, you can call the function 'describe', as shown below:

In [ ]:
all_precip[["actual_precipitation_seattle","actual_precipitation_philly"]].describe()

We can also use `pandas` (and `matplotlib` which you'll learn more about in a future lesson) to help plot our data. 

Let's plot the record precipitation in Philadelphia vs. Seattle. In this case, we'll use a line plot to show the data over time.

In [ ]:
# Plot the Philly daily record precipitation in purple
all_precip.plot(kind='line', y='record_precipitation_philly', ax=plt.gca(), color='purple', figsize=(12,5))

# Plot the Seattle daily record precipitation in green
all_precip.plot(kind='line', y='record_precipitation_seattle', ax=plt.gca(), color='green', figsize=(12,5))

# Display the plot
# Note using plt.gca() to get the current axes allows us to plot multiple lines on the same graph
plt.show()

From this graph, we can see that months where the record precipitation was on average the highest in Philadelphia (such as August and July) are the lowest precipitation times in Seattle!

### In-Class Exercises <a id='exercises'></a>

1. Use label-based subsetting to select the precipitation data from July 1st and July 2nd, 2014, in Philadelphia.

In [ ]:
philly_weather.loc[["2014-7-1", "2014-7-2"], ["actual_precipitation", "average_precipitation", "record_precipitation"]]

2. Use index-based subsetting to select the same precipitation data as in Exercise 1.

In [ ]:
philly_weather.iloc[0:2, 9:12]

3. Use the philly_weather and seattle_weather objects to figure out which days had a record minimum tempeature under 10 in either dataset.

In [ ]:
# Option 1
all_weather = philly_weather.join(seattle_weather, lsuffix='_philly', rsuffix='_seattle')
all_weather[(all_weather["record_min_temp_philly"] < 10) | (all_weather["record_min_temp_seattle"] < 10)]

# Option 2
low_philly = philly_weather[philly_weather["record_min_temp"] < 10]
low_seattle = seattle_weather[seattle_weather["record_min_temp"] < 10]
low_philly.join(low_seattle, lsuffix='_philly', rsuffix='_seattle')

4. Using the `all_precip` object, find the difference in both the mean and standard deviation of average precipitation between Philadelphia and Seattle.

In [ ]:
mean_diff = all_precip["average_precipitation_philly"].mean() - all_precip["average_precipitation_seattle"].mean()
print(f'The difference in means is {mean_diff:.3f}.')

std_diff = all_precip["average_precipitation_philly"].std() - all_precip["average_precipitation_seattle"].std()
print(f'The difference in standard deviations is {std_diff:.3f}.')